# Netflix Movie/Series Recommendation System USING NLP


In this project i tried to build a recommendation system just like netflix.

To personalize the experience we will require 
the data of user and a user id so that we can add a separate feature of recommendation as per the shows/movies they have already rated.

The recommendation that we are giving will be based on the similarity in genre of movies. It can also be based on the cast of movie or the director of movie.

{{Country states that the content ws taken from which country}}

In [1]:
import pandas as pd
import numpy as np

In [2]:
#reading all the data frames and making duplicates
df=pd.read_excel(r"D:\CV\Projects present in CV\Netflix recommendation system/netflix_titles.xlsx")
df1=df
df2=df
df3=df
df4=df
df5=df
df6=df
df7=df
df8=df
df9=df
df10=df
df11=df
#df.head(2)

In [3]:
#taking input from user 
k=input("Movie/show name from netfix: ")

Movie/show name from netfix: Friends


First we will distribute the movies as per genre  
HERE WE ARE USING NLP TO CREATE A VECTOR OUT OF GENRE SENTENCE

In [4]:
#replacing nan values with blanks so that nan doesnot hinder in NLP
df = df.replace(np.nan, '', regex=True)

In [5]:
df.head(2)

,title,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Norm of the North: King Sized Adventure,Movie,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09 00:00:00,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Movie,,Jandino Asporaat,United Kingdom,2016-09-09 00:00:00,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...


In [6]:
#converting data to strings format so that NLP can be applied
df['release_year'] = df['release_year'].astype(str)
df['duration'] = df['duration'].astype(str)
df['date_added'] = df['date_added'].astype(str)

# CASE 1

IN CASE 1 {WE TOOK ALL THE CLUMNS- TITLE, TYPE, DIRECTOR, CAST, COUNTRY, DATE_ADDED, RELEASE_YEAR, RATING, DURATION, LISTED_IN, DESCRIPTION}

In [7]:
#creating a new column by adding the strings separated by spaces from all the columns 
df["model1"] = df["type"] +" "+ df["director"]+" "+ df["cast"]+" "+ df["country"]+" "+ df["release_year"]+" "+ df["rating"]+" "+ df["duration"]+" "+ df["listed_in"]+" "+ df["description"]

In [8]:
#df.head(2)

In [9]:
#taking titile and model1 column
df=df.iloc[:,[0,11]]
df.head(2)

,title,model1
0,Norm of the North: King Sized Adventure,"Movie Richard Finn, Tim Maltby Alan Marriott, ..."
1,Jandino: Whatever it Takes,Movie Jandino Asporaat United Kingdom 2016 TV...


In [10]:
#NLP
#from sklearn lib . module and imported class
#we are taking a sentence and converting it to a vector so that we can draw some inference from it by seeking  out
#the words that are repeating
#tfidf is a technique which helps us to create document matrix from sentences


from sklearn.feature_extraction.text import TfidfVectorizer

#ngram_range 1-3 means we are taking a combination of 3 different words and we are stopping english words like it the is 
#an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, 
#and (2, 2) means only bigrams. Only applies if analyzer is not callable.

#max_features parameter    -- If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus. 
#This parameter is ignored if vocabulary is not None.

#min dif When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. 
#This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents,
#integer absolute counts. This parameter is ignored if vocabulary is not None.

#strip_accents{‘ascii’, ‘unicode’, None} (default=None)
#Remove accents and perform other character normalization during the preprocessing step. 
#‘ascii’ is a fast method that only works on characters that have an direct ASCII mapping. 
#‘unicode’ is a slightly slower method that works on any characters. None (default) does nothing.


#analyzerstr, {‘word’, ‘char’, ‘char_wb’} or callable
#Whether the feature should be made of word or character n-grams. 
#Option ‘char_wb’ creates character n-grams only from text inside word boundaries; n-grams at 
#the edges of words are padded with space.
#If a callable is passed it is used to extract the sequence of features out of the raw, unprocessed input

#token_patternstr
#Regular expression denoting what constitutes a “token”, only used if analyzer == 'word'. 
#The default regexp selects tokens of 2 or more alphanumeric characters (punctuation is completely 
#ignored and always treated as a token separator).


tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

# Filling NaNs with empty string
df['model1'] = df['model1'].fillna('')

In [11]:
#a new matrix is created and the model is fitted to it and column is also stated
tfv_matrix = tfv.fit_transform(df['model1'])

In [12]:
#a matrix with vectors is created
tfv_matrix

<6234x26456 sparse matrix of type '<class 'numpy.float64'>'
	with 373561 stored elements in Compressed Sparse Row format>

In [13]:
#sigmoid kernel is used in logistics regression 
#we get a curve bw 0 and 1 so it transforms output to 0 and 1

from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [14]:
sig[0]

array([0.76161003, 0.76159424, 0.76159432, ..., 0.76159435, 0.76159442,
       0.76159462])

In [15]:
#the title column has been set to indices
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [16]:
indices

title
Norm of the North: King Sized Adventure           0
Jandino: Whatever it Takes                        1
Transformers Prime                                2
Transformers: Robots in Disguise                  3
#realityhigh                                      4
                                               ... 
Red vs. Blue                                   6229
Maron                                          6230
Little Baby Bum: Nursery Rhyme Friends         6231
A Young Doctor's Notebook and Other Stories    6232
Friends                                        6233
Length: 6234, dtype: int64

In [17]:
indices['Friends']

6233

In [18]:
#this is the vector for friends
sig[4799]

array([0.76159455, 0.76159478, 0.76159451, ..., 0.76159416, 0.76159515,
       0.76159456])

In [19]:
list(enumerate(sig[indices['Friends']]))

[(0, 0.7615946177164992),
 (1, 0.7615942561171928),
 (2, 0.7615943439046888),
 (3, 0.7615943753451799),
 (4, 0.761594371917185),
 (5, 0.7615944249786826),
 (6, 0.7615942998872225),
 (7, 0.761594201138133),
 (8, 0.7615944468094663),
 (9, 0.7615942447205484),
 (10, 0.7615942002179181),
 (11, 0.7615943202709607),
 (12, 0.7615941964803105),
 (13, 0.7615941913364355),
 (14, 0.7615941924955649),
 (15, 0.7615941939777243),
 (16, 0.7615942017960373),
 (17, 0.7615941991017435),
 (18, 0.7615941931892183),
 (19, 0.7615943402671201),
 (20, 0.7615948284949201),
 (21, 0.7615941929432212),
 (22, 0.7615946306843313),
 (23, 0.7615942946430254),
 (24, 0.7615942235882363),
 (25, 0.761594412980644),
 (26, 0.76159483185279),
 (27, 0.7615943147133892),
 (28, 0.7615942870923831),
 (29, 0.7615942000815059),
 (30, 0.761594372295989),
 (31, 0.7615942544180844),
 (32, 0.7615942885973617),
 (33, 0.7615941559557649),
 (34, 0.7615950199743577),
 (35, 0.7615941935073005),
 (36, 0.761594437848302),
 (37, 0.7615943303

In [20]:
#the vector is sorted
sorted(list(enumerate(sig[indices['Friends']])), key=lambda x: x[1], reverse=True)

[(6233, 0.761610029944723),
 (6225, 0.7615990913090078),
 (5987, 0.7615989311501604),
 (5830, 0.7615977808561074),
 (5980, 0.7615975150885762),
 (5981, 0.7615973681817615),
 (5787, 0.7615971164150193),
 (5699, 0.7615970426282657),
 (6094, 0.7615969131263384),
 (5976, 0.7615967893225191),
 (5639, 0.7615967665890371),
 (5585, 0.761596718788903),
 (5704, 0.7615967012158968),
 (5705, 0.7615967010146079),
 (6072, 0.7615966462496775),
 (5636, 0.7615965823873305),
 (6166, 0.7615965765628401),
 (1827, 0.7615963278098152),
 (1600, 0.7615962991420286),
 (5843, 0.7615960391116637),
 (786, 0.761596019537951),
 (5909, 0.7615959232397969),
 (6179, 0.7615959217717962),
 (5982, 0.7615958516194855),
 (3390, 0.7615957220978671),
 (5669, 0.7615956108031461),
 (4470, 0.7615955554764072),
 (5725, 0.7615955437681702),
 (1282, 0.7615955384394869),
 (5582, 0.7615955302535835),
 (1263, 0.7615955101899872),
 (648, 0.7615955029240398),
 (3456, 0.7615954380486082),
 (5750, 0.7615954345403492),
 (3109, 0.761595413

In [21]:
def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:21]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return df['title'].iloc[movie_indices]

In [22]:
give_rec(k)

6225                                Frasier
5987                               Episodes
5830                             Dad's Army
5980                 The Andy Griffith Show
5981    The Twilight Zone (Original Series)
5787                           Supernatural
5699                             Still Game
6094                    Pee-wee's Playhouse
5976                                 Cheers
5639                        Toast of London
5585                                  Haven
5704                              Star Trek
5705                  Star Trek: Enterprise
6072                    From Dusk Till Dawn
5636                       The Walking Dead
6166                                Jericho
1827                    Roswell, New Mexico
1600                   PasiÃ³n de Gavilanes
5843                      Trailer Park Boys
786         Charlie's Angels: Full Throttle
Name: title, dtype: object

In [23]:
case1=list(give_rec(k))

In [24]:
case1

['Frasier',
 'Episodes',
 "Dad's Army",
 'The Andy Griffith Show',
 'The Twilight Zone (Original Series)',
 'Supernatural',
 'Still Game',
 "Pee-wee's Playhouse",
 'Cheers',
 'Toast of London',
 'Haven',
 'Star Trek',
 'Star Trek: Enterprise',
 'From Dusk Till Dawn',
 'The Walking Dead',
 'Jericho',
 'Roswell, New Mexico',
 'PasiÃ³n de Gavilanes',
 'Trailer Park Boys',
 "Charlie's Angels: Full Throttle"]

# CASE2

LISTED IN, Type

In [25]:
df1.head(2)

,title,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Norm of the North: King Sized Adventure,Movie,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09 00:00:00,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Movie,NaN,Jandino Asporaat,United Kingdom,2016-09-09 00:00:00,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...


In [26]:
df1 = df1.replace(np.nan, '', regex=True)
df1['release_year'] = df1['release_year'].astype(str)
df1['duration'] = df1['duration'].astype(str)
df1['date_added'] = df1['date_added'].astype(str)

In [27]:
df1["model1"] = df1["type"]+" "+ df1["listed_in"]

In [28]:
df1=df1.iloc[:,[0,11]]
df.head()

,title,model1
0,Norm of the North: King Sized Adventure,"Movie Richard Finn, Tim Maltby Alan Marriott, ..."
1,Jandino: Whatever it Takes,Movie Jandino Asporaat United Kingdom 2016 TV...
2,Transformers Prime,"TV Show Peter Cullen, Sumalee Montano, Frank ..."
3,Transformers: Robots in Disguise,"TV Show Will Friedle, Darren Criss, Constance..."
4,#realityhigh,"Movie Fernando Lebrija Nesta Cooper, Kate Wals..."


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

# Filling NaNs with empty string
df1['model1'] = df1['model1'].fillna('')

In [30]:
tfv_matrix = tfv.fit_transform(df1['model1'])

In [31]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [32]:
indices = pd.Series(df1.index, index=df1['title']).drop_duplicates()

In [33]:
def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:21]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return df1['title'].iloc[movie_indices]

In [34]:
give_rec(k)

5980                 The Andy Griffith Show
5987                               Episodes
6225                                Frasier
6233                                Friends
6094                    Pee-wee's Playhouse
5981    The Twilight Zone (Original Series)
5636                       The Walking Dead
6166                                Jericho
5787                           Supernatural
6072                    From Dusk Till Dawn
5585                                  Haven
1827                    Roswell, New Mexico
5704                              Star Trek
5705                  Star Trek: Enterprise
5982                             Twin Peaks
5639                        Toast of London
5699                             Still Game
5830                             Dad's Army
3390                   La Rosa de Guadalupe
5843                      Trailer Park Boys
Name: title, dtype: object

In [35]:
case2=list(give_rec(k))

In [36]:
case2

['The Andy Griffith Show',
 'Episodes',
 'Frasier',
 'Friends',
 "Pee-wee's Playhouse",
 'The Twilight Zone (Original Series)',
 'The Walking Dead',
 'Jericho',
 'Supernatural',
 'From Dusk Till Dawn',
 'Haven',
 'Roswell, New Mexico',
 'Star Trek',
 'Star Trek: Enterprise',
 'Twin Peaks',
 'Toast of London',
 'Still Game',
 "Dad's Army",
 'La Rosa de Guadalupe',
 'Trailer Park Boys']

# Case3

Country, release_year, listed_in

In [37]:
df2.head(2)

,title,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Norm of the North: King Sized Adventure,Movie,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09 00:00:00,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Movie,NaN,Jandino Asporaat,United Kingdom,2016-09-09 00:00:00,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...


In [38]:
df2 = df2.replace(np.nan, '', regex=True)
df2 = df2.replace(np.nan, '', regex=True)
df2['release_year'] = df2['release_year'].astype(str)
df2['duration'] = df2['duration'].astype(str)
df2['date_added'] = df2['date_added'].astype(str)

In [39]:
df2["model1"] = df2["country"]+" "+ df2["listed_in"]+" "+ df2["release_year"]

In [40]:
df2=df2.iloc[:,[0,11]]

In [41]:
df2.head()

,title,model1
0,Norm of the North: King Sized Adventure,"United States, India, South Korea, China Child..."
1,Jandino: Whatever it Takes,United Kingdom Stand-Up Comedy 2016
2,Transformers Prime,United States Kids' TV 2013
3,Transformers: Robots in Disguise,United States Kids' TV 2016
4,#realityhigh,United States Comedies 2017


In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

# Filling NaNs with empty string
df2['model1'] = df2['model1'].fillna('')

In [43]:
tfv_matrix = tfv.fit_transform(df2['model1'])

In [44]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()



In [45]:
def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:21]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return df2['title'].iloc[movie_indices]

In [46]:
give_rec(k)

6233                                    Friends
5980                     The Andy Griffith Show
5976                                     Cheers
6094                        Pee-wee's Playhouse
5981        The Twilight Zone (Original Series)
5636                           The Walking Dead
5585                                      Haven
6166                                    Jericho
5787                               Supernatural
1827                        Roswell, New Mexico
6072                        From Dusk Till Dawn
5704                                  Star Trek
5987                                   Episodes
5705                      Star Trek: Enterprise
5830                                 Dad's Army
5982                                 Twin Peaks
5699                                 Still Game
5639                            Toast of London
5725                                   Kopitiam
791     Dumb and Dumberer: When Harry Met Lloyd
Name: title, dtype: object

In [47]:
case3=list(give_rec(k))

In [48]:
case3

['Friends',
 'The Andy Griffith Show',
 'Cheers',
 "Pee-wee's Playhouse",
 'The Twilight Zone (Original Series)',
 'The Walking Dead',
 'Haven',
 'Jericho',
 'Supernatural',
 'Roswell, New Mexico',
 'From Dusk Till Dawn',
 'Star Trek',
 'Episodes',
 'Star Trek: Enterprise',
 "Dad's Army",
 'Twin Peaks',
 'Still Game',
 'Toast of London',
 'Kopitiam',
 'Dumb and Dumberer: When Harry Met Lloyd']

# Case4

listed-in, Cast

In [49]:
df3.head(2)

,title,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Norm of the North: King Sized Adventure,Movie,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09 00:00:00,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Movie,NaN,Jandino Asporaat,United Kingdom,2016-09-09 00:00:00,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...


In [50]:
df3 = df3.replace(np.nan, '', regex=True)
df3['release_year'] = df3['release_year'].astype(str)
df3['duration'] = df3['duration'].astype(str)
df3['date_added'] = df3['date_added'].astype(str)
df3["model1"] = df3["cast"]+" "+ df3["listed_in"]
df3=df3.iloc[:,[0,11]]
df3.head(2)

,title,model1
0,Norm of the North: King Sized Adventure,"Alan Marriott, Andrew Toth, Brian Dobson, Cole..."
1,Jandino: Whatever it Takes,Jandino Asporaat Stand-Up Comedy


In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

# Filling NaNs with empty string
df3['model1'] = df3['model1'].fillna('')

In [52]:
tfv_matrix = tfv.fit_transform(df3['model1'])

In [53]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

indices = pd.Series(df3.index, index=df3['title']).drop_duplicates()

In [54]:
def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:21]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return df3['title'].iloc[movie_indices]

In [55]:
give_rec(k)

5987                               Episodes
6094                    Pee-wee's Playhouse
5981    The Twilight Zone (Original Series)
5980                 The Andy Griffith Show
5639                        Toast of London
5699                             Still Game
6225                                Frasier
5830                             Dad's Army
5705                  Star Trek: Enterprise
5787                           Supernatural
5976                                 Cheers
5704                              Star Trek
5585                                  Haven
1827                    Roswell, New Mexico
701                          Harry & Bunnie
1198                           Paap-O-Meter
4363                             Sammy & Co
5479                   Kicko & Super Speedo
5778            Prince Jai Aur Dumdaar Viru
5867                       Sab Jholmaal Hai
Name: title, dtype: object

In [56]:
case4=list(give_rec(k))

In [57]:
case4

['Episodes',
 "Pee-wee's Playhouse",
 'The Twilight Zone (Original Series)',
 'The Andy Griffith Show',
 'Toast of London',
 'Still Game',
 'Frasier',
 "Dad's Army",
 'Star Trek: Enterprise',
 'Supernatural',
 'Cheers',
 'Star Trek',
 'Haven',
 'Roswell, New Mexico',
 'Harry & Bunnie',
 'Paap-O-Meter',
 'Sammy & Co',
 'Kicko & Super Speedo',
 'Prince Jai Aur Dumdaar Viru',
 'Sab Jholmaal Hai']

# Case 5

listedin, duration

In [58]:
df4.head(2)

,title,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Norm of the North: King Sized Adventure,Movie,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09 00:00:00,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Movie,NaN,Jandino Asporaat,United Kingdom,2016-09-09 00:00:00,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...


In [59]:
df4 = df4.replace(np.nan, '', regex=True)
df4['release_year'] = df4['release_year'].astype(str)
df4['duration'] = df4['duration'].astype(str)
df4['date_added'] = df4['date_added'].astype(str)
df4["model1"] = df4["duration"]+" "+ df4["listed_in"]
df4=df4.iloc[:,[0,11]]
df4.head(2)

,title,model1
0,Norm of the North: King Sized Adventure,"90 min Children & Family Movies, Comedies"
1,Jandino: Whatever it Takes,94 min Stand-Up Comedy


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

# Filling NaNs with empty string
df4['model1'] = df4['model1'].fillna('')


In [61]:
tfv_matrix = tfv.fit_transform(df4['model1'])

In [62]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

indices = pd.Series(df4.index, index=df4['title']).drop_duplicates()

In [63]:
def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:21]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return df4['title'].iloc[movie_indices]

In [64]:
give_rec(k)

5980                 The Andy Griffith Show
5976                                 Cheers
6225                                Frasier
5987                               Episodes
5830                             Dad's Army
6094                    Pee-wee's Playhouse
5981    The Twilight Zone (Original Series)
6166                                Jericho
6072                    From Dusk Till Dawn
5636                       The Walking Dead
5787                           Supernatural
5585                                  Haven
5704                              Star Trek
5705                  Star Trek: Enterprise
5639                        Toast of London
5982                             Twin Peaks
5699                             Still Game
5843                      Trailer Park Boys
5909       Danger Mouse: Classic Collection
1827                    Roswell, New Mexico
Name: title, dtype: object

In [65]:
case5=list(give_rec(k))

In [66]:
case5

['The Andy Griffith Show',
 'Cheers',
 'Frasier',
 'Episodes',
 "Dad's Army",
 "Pee-wee's Playhouse",
 'The Twilight Zone (Original Series)',
 'Jericho',
 'From Dusk Till Dawn',
 'The Walking Dead',
 'Supernatural',
 'Haven',
 'Star Trek',
 'Star Trek: Enterprise',
 'Toast of London',
 'Twin Peaks',
 'Still Game',
 'Trailer Park Boys',
 'Danger Mouse: Classic Collection',
 'Roswell, New Mexico']

# Case-6

Cast,director

In [67]:
df5.head(2)

,title,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Norm of the North: King Sized Adventure,Movie,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09 00:00:00,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Movie,NaN,Jandino Asporaat,United Kingdom,2016-09-09 00:00:00,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...


In [68]:
df5 = df5.replace(np.nan, '', regex=True)
df5['release_year'] = df5['release_year'].astype(str)
df5['duration'] = df5['duration'].astype(str)
df5['date_added'] = df5['date_added'].astype(str)
df5["model1"] = df5["cast"]+" "+ df5["director"]
df5=df5.iloc[:,[0,11]]
df5.head(2)

,title,model1
0,Norm of the North: King Sized Adventure,"Alan Marriott, Andrew Toth, Brian Dobson, Cole..."
1,Jandino: Whatever it Takes,Jandino Asporaat


In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

# Filling NaNs with empty string
df5['model1'] = df5['model1'].fillna('')

In [70]:
tfv_matrix = tfv.fit_transform(df5['model1'])

In [71]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

indices = pd.Series(df5.index, index=df5['title']).drop_duplicates()

In [72]:
def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:21]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return df5['title'].iloc[movie_indices]

In [73]:
give_rec(k)

5987                                    Episodes
1703    Highland: Thailand's Marijuana Awakening
3083                 The Accountant of Auschwitz
587                          Miss Representation
886                                     Mountain
2418                                 Sustainable
3923                                   Studio 54
5482                             P.S. I Love You
3456                                        Zoom
5395                          The Creative Brain
3074                                    Scream 3
5076                                Rumor Has It
785                             Charlie's Angels
2662                              Murder Mystery
1591            A Little Help with Carol Burnett
3412                                      Unrest
786              Charlie's Angels: Full Throttle
1976                                  My Own Man
2764                              The Fear of 13
4483                              Theo Who Lived
Name: title, dtype: 

In [74]:
case6=list(give_rec(k))
case6

['Episodes',
 "Highland: Thailand's Marijuana Awakening",
 'The Accountant of Auschwitz',
 'Miss Representation',
 'Mountain',
 'Sustainable',
 'Studio 54',
 'P.S. I Love You',
 'Zoom',
 'The Creative Brain',
 'Scream 3',
 'Rumor Has It',
 "Charlie's Angels",
 'Murder Mystery',
 'A Little Help with Carol Burnett',
 'Unrest',
 "Charlie's Angels: Full Throttle",
 'My Own Man',
 'The Fear of 13',
 'Theo Who Lived']

# Case-7

Listed_in,country

In [75]:
df6.head(2)

,title,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Norm of the North: King Sized Adventure,Movie,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09 00:00:00,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Movie,NaN,Jandino Asporaat,United Kingdom,2016-09-09 00:00:00,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...


In [76]:
df6 = df6.replace(np.nan, '', regex=True)
df6['release_year'] = df6['release_year'].astype(str)
df6['duration'] = df6['duration'].astype(str)
df6['date_added'] = df6['date_added'].astype(str)
df6["model1"] = df6["country"]+" "+ df6["listed_in"]
df6=df6.iloc[:,[0,11]]
df6.head(2)

,title,model1
0,Norm of the North: King Sized Adventure,"United States, India, South Korea, China Child..."
1,Jandino: Whatever it Takes,United Kingdom Stand-Up Comedy


In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

# Filling NaNs with empty string
df6['model1'] = df6['model1'].fillna('')

In [78]:
tfv_matrix = tfv.fit_transform(df6['model1'])

In [79]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

indices = pd.Series(df6.index, index=df6['title']).drop_duplicates()

In [80]:
def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:21]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return df6['title'].iloc[movie_indices]

In [81]:
give_rec(k)

5980                 The Andy Griffith Show
6225                                Frasier
6233                                Friends
6094                    Pee-wee's Playhouse
5981    The Twilight Zone (Original Series)
5636                       The Walking Dead
6166                                Jericho
5787                           Supernatural
6072                    From Dusk Till Dawn
5585                                  Haven
1827                    Roswell, New Mexico
5704                              Star Trek
5705                  Star Trek: Enterprise
5987                               Episodes
5982                             Twin Peaks
5639                        Toast of London
5699                             Still Game
5830                             Dad's Army
5669           Monty Python's Flying Circus
6179                           The IT Crowd
Name: title, dtype: object

In [82]:
case7=list(give_rec(k))
case7

['The Andy Griffith Show',
 'Frasier',
 'Friends',
 "Pee-wee's Playhouse",
 'The Twilight Zone (Original Series)',
 'The Walking Dead',
 'Jericho',
 'Supernatural',
 'From Dusk Till Dawn',
 'Haven',
 'Roswell, New Mexico',
 'Star Trek',
 'Star Trek: Enterprise',
 'Episodes',
 'Twin Peaks',
 'Toast of London',
 'Still Game',
 "Dad's Army",
 "Monty Python's Flying Circus",
 'The IT Crowd']

# Case-8 

Listed-in,description

In [83]:
df7.head(2)

,title,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Norm of the North: King Sized Adventure,Movie,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09 00:00:00,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Movie,NaN,Jandino Asporaat,United Kingdom,2016-09-09 00:00:00,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...


In [84]:
df7 = df7.replace(np.nan, '', regex=True)
df7['release_year'] = df7['release_year'].astype(str)
df7['duration'] = df7['duration'].astype(str)
df7['date_added'] = df7['date_added'].astype(str)
df7["model1"] = df7["description"]+" "+ df7["listed_in"]
df7=df7.iloc[:,[0,11]]
df7.head(2)

,title,model1
0,Norm of the North: King Sized Adventure,Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Jandino Asporaat riffs on the challenges of ra...


In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

# Filling NaNs with empty string
df7['model1'] = df7['model1'].fillna('')

In [86]:
tfv_matrix = tfv.fit_transform(df7['model1'])

In [87]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

indices = pd.Series(df7.index, index=df7['title']).drop_duplicates()

In [88]:
def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:21]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return df7['title'].iloc[movie_indices]

In [89]:
give_rec(k)

5987                               Episodes
6225                                Frasier
5830                             Dad's Army
5699                             Still Game
5980                 The Andy Griffith Show
5976                                 Cheers
5639                        Toast of London
5981    The Twilight Zone (Original Series)
5636                       The Walking Dead
6094                    Pee-wee's Playhouse
6072                    From Dusk Till Dawn
6166                                Jericho
5843                      Trailer Park Boys
5787                           Supernatural
5585                                  Haven
5704                              Star Trek
1827                    Roswell, New Mexico
5705                  Star Trek: Enterprise
3390                   La Rosa de Guadalupe
6179                           The IT Crowd
Name: title, dtype: object

In [90]:
case8=list(give_rec(k))
case8

['Episodes',
 'Frasier',
 "Dad's Army",
 'Still Game',
 'The Andy Griffith Show',
 'Cheers',
 'Toast of London',
 'The Twilight Zone (Original Series)',
 'The Walking Dead',
 "Pee-wee's Playhouse",
 'From Dusk Till Dawn',
 'Jericho',
 'Trailer Park Boys',
 'Supernatural',
 'Haven',
 'Star Trek',
 'Roswell, New Mexico',
 'Star Trek: Enterprise',
 'La Rosa de Guadalupe',
 'The IT Crowd']

# Case-9

listedin,realese_year

In [91]:
df8.head(2)

,title,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Norm of the North: King Sized Adventure,Movie,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09 00:00:00,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Movie,NaN,Jandino Asporaat,United Kingdom,2016-09-09 00:00:00,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...


In [92]:
df8 = df8.replace(np.nan, '', regex=True)
df8['release_year'] = df8['release_year'].astype(str)
df8['duration'] = df8['duration'].astype(str)
df8['date_added'] = df8['date_added'].astype(str)
df8["model1"] = df8["release_year"]+" "+ df8["listed_in"]
df8=df8.iloc[:,[0,11]]
df8.head(2)

,title,model1
0,Norm of the North: King Sized Adventure,"2019 Children & Family Movies, Comedies"
1,Jandino: Whatever it Takes,2016 Stand-Up Comedy


In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3),
            stop_words = 'english')

# Filling NaNs with empty string
df8['model1'] = df8['model1'].fillna('')

In [94]:
tfv_matrix = tfv.fit_transform(df8['model1'])

In [95]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

indices = pd.Series(df8.index, index=df8['title']).drop_duplicates()

In [96]:
def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies
    sig_scores = sig_scores[1:21]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return df8['title'].iloc[movie_indices]

In [97]:
give_rec(k)

6233                                Friends
5987                               Episodes
5976                                 Cheers
5980                 The Andy Griffith Show
5981    The Twilight Zone (Original Series)
6094                    Pee-wee's Playhouse
5830                             Dad's Army
5699                             Still Game
5639                        Toast of London
5636                       The Walking Dead
5585                                  Haven
6166                                Jericho
5787                           Supernatural
6072                    From Dusk Till Dawn
1827                    Roswell, New Mexico
5705                  Star Trek: Enterprise
5704                              Star Trek
5669           Monty Python's Flying Circus
5843                      Trailer Park Boys
3390                   La Rosa de Guadalupe
Name: title, dtype: object

In [98]:
case9=list(give_rec(k))
case9

['Friends',
 'Episodes',
 'Cheers',
 'The Andy Griffith Show',
 'The Twilight Zone (Original Series)',
 "Pee-wee's Playhouse",
 "Dad's Army",
 'Still Game',
 'Toast of London',
 'The Walking Dead',
 'Haven',
 'Jericho',
 'Supernatural',
 'From Dusk Till Dawn',
 'Roswell, New Mexico',
 'Star Trek: Enterprise',
 'Star Trek',
 "Monty Python's Flying Circus",
 'Trailer Park Boys',
 'La Rosa de Guadalupe']

# Case-10

Listedin,description,rating

In [99]:
df9.head(2)

,title,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Norm of the North: King Sized Adventure,Movie,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09 00:00:00,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Movie,NaN,Jandino Asporaat,United Kingdom,2016-09-09 00:00:00,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...


In [100]:
df9 = df9.replace(np.nan, '', regex=True)
df9['release_year'] = df9['release_year'].astype(str)
df9['duration'] = df9['duration'].astype(str)
df9['date_added'] = df9['date_added'].astype(str)
df9["model1"] = df9["description"]+" "+ df9["listed_in"]+" "+ df9["rating"]
df9=df9.iloc[:,[0,11]]
df9.head(2)

,title,model1
0,Norm of the North: King Sized Adventure,Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Jandino Asporaat riffs on the challenges of ra...


In [101]:
tfv_matrix = tfv.fit_transform(df9['model1'])

In [102]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

indices = pd.Series(df9.index, index=df9['title']).drop_duplicates()

In [103]:
def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies/shows
    sig_scores = sig_scores[1:21]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return df9['title'].iloc[movie_indices]

In [104]:
give_rec(k)

5987                               Episodes
6225                                Frasier
5699                             Still Game
5830                             Dad's Army
5980                 The Andy Griffith Show
5976                                 Cheers
5639                        Toast of London
6094                    Pee-wee's Playhouse
5636                       The Walking Dead
6166                                Jericho
5981    The Twilight Zone (Original Series)
5843                      Trailer Park Boys
5585                                  Haven
6072                    From Dusk Till Dawn
5787                           Supernatural
1827                    Roswell, New Mexico
5704                              Star Trek
5705                  Star Trek: Enterprise
3390                   La Rosa de Guadalupe
6179                           The IT Crowd
Name: title, dtype: object

In [105]:
case10=list(give_rec(k))
case10

['Episodes',
 'Frasier',
 'Still Game',
 "Dad's Army",
 'The Andy Griffith Show',
 'Cheers',
 'Toast of London',
 "Pee-wee's Playhouse",
 'The Walking Dead',
 'Jericho',
 'The Twilight Zone (Original Series)',
 'Trailer Park Boys',
 'Haven',
 'From Dusk Till Dawn',
 'Supernatural',
 'Roswell, New Mexico',
 'Star Trek',
 'Star Trek: Enterprise',
 'La Rosa de Guadalupe',
 'The IT Crowd']

# Case 11

Listedin, country,rating

In [106]:
df10.head(2)

,title,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Norm of the North: King Sized Adventure,Movie,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09 00:00:00,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Movie,NaN,Jandino Asporaat,United Kingdom,2016-09-09 00:00:00,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...


In [107]:
df10 = df10.replace(np.nan, '', regex=True)
df10['release_year'] = df10['release_year'].astype(str)
df10['duration'] = df10['duration'].astype(str)
df10['date_added'] = df10['date_added'].astype(str)
df10["model1"] = df10["country"]+" "+ df10["listed_in"]+" "+ df10["rating"]
df10=df10.iloc[:,[0,11]]
df10.head(2)

,title,model1
0,Norm of the North: King Sized Adventure,"United States, India, South Korea, China Child..."
1,Jandino: Whatever it Takes,United Kingdom Stand-Up Comedy TV-MA


In [108]:
tfv_matrix = tfv.fit_transform(df10['model1'])

from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

indices = pd.Series(df10.index, index=df10['title']).drop_duplicates()

def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies/shows
    sig_scores = sig_scores[1:21]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return df10['title'].iloc[movie_indices]

In [109]:
give_rec(k)

5976                                 Cheers
6225                                Frasier
5980                 The Andy Griffith Show
6094                    Pee-wee's Playhouse
6166                                Jericho
5636                       The Walking Dead
5981    The Twilight Zone (Original Series)
1827                    Roswell, New Mexico
5585                                  Haven
5787                           Supernatural
5699                             Still Game
5987                               Episodes
6072                    From Dusk Till Dawn
5704                              Star Trek
5705                  Star Trek: Enterprise
5639                        Toast of London
5982                             Twin Peaks
5830                             Dad's Army
5938                           Kath and Kim
4946                       Magic for Humans
Name: title, dtype: object

In [110]:
case11=list(give_rec(k))
case11

['Cheers',
 'Frasier',
 'The Andy Griffith Show',
 "Pee-wee's Playhouse",
 'Jericho',
 'The Walking Dead',
 'The Twilight Zone (Original Series)',
 'Roswell, New Mexico',
 'Haven',
 'Supernatural',
 'Still Game',
 'Episodes',
 'From Dusk Till Dawn',
 'Star Trek',
 'Star Trek: Enterprise',
 'Toast of London',
 'Twin Peaks',
 "Dad's Army",
 'Kath and Kim',
 'Magic for Humans']

# Case 12

Listedin,country,rating,release_year

In [111]:
df11.head(2)

,title,type,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,Norm of the North: King Sized Adventure,Movie,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09 00:00:00,2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,Jandino: Whatever it Takes,Movie,NaN,Jandino Asporaat,United Kingdom,2016-09-09 00:00:00,2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...


In [112]:
df11 = df11.replace(np.nan, '', regex=True)
df11['release_year'] = df11['release_year'].astype(str)
df11['duration'] = df11['duration'].astype(str)
df11['date_added'] = df11['date_added'].astype(str)
df11["model1"] = df11["country"]+" "+ df11["listed_in"]+" "+ df11["rating"]+" "+ df11["release_year"]
df11=df11.iloc[:,[0,11]]
df11.head(2)

,title,model1
0,Norm of the North: King Sized Adventure,"United States, India, South Korea, China Child..."
1,Jandino: Whatever it Takes,United Kingdom Stand-Up Comedy TV-MA 2016


In [113]:
tfv_matrix = tfv.fit_transform(df11['model1'])

from sklearn.metrics.pairwise import sigmoid_kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

indices = pd.Series(df11.index, index=df11['title']).drop_duplicates()

def give_rec(title, sig=sig):
    # Get the index corresponding to original_title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(sig[idx]))

    # Sort the movies 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar movies/shows
    sig_scores = sig_scores[1:21]

    # Movie indices
    movie_indices = [i[0] for i in sig_scores]

    # Top 10 most similar movies
    return df11['title'].iloc[movie_indices]

In [114]:
give_rec(k)

6225                                Frasier
5980                 The Andy Griffith Show
5976                                 Cheers
6094                    Pee-wee's Playhouse
5981    The Twilight Zone (Original Series)
5636                       The Walking Dead
6166                                Jericho
1827                    Roswell, New Mexico
5787                           Supernatural
5585                                  Haven
5699                             Still Game
6072                    From Dusk Till Dawn
5987                               Episodes
5725                               Kopitiam
5704                              Star Trek
5705                  Star Trek: Enterprise
5639                        Toast of London
5830                             Dad's Army
5982                             Twin Peaks
1750                           Kucch To Hai
Name: title, dtype: object

In [115]:
case12=(give_rec(k))
case12

6225                                Frasier
5980                 The Andy Griffith Show
5976                                 Cheers
6094                    Pee-wee's Playhouse
5981    The Twilight Zone (Original Series)
5636                       The Walking Dead
6166                                Jericho
1827                    Roswell, New Mexico
5787                           Supernatural
5585                                  Haven
5699                             Still Game
6072                    From Dusk Till Dawn
5987                               Episodes
5725                               Kopitiam
5704                              Star Trek
5705                  Star Trek: Enterprise
5639                        Toast of London
5830                             Dad's Army
5982                             Twin Peaks
1750                           Kucch To Hai
Name: title, dtype: object

In [116]:
data = pd.DataFrame(list(zip(case1, case2, case3,case4,case5,case6,case7,case8,case9,case10,case11,case12)), 
               columns =['case1', 'case2', 'case3','case4','case5','case6','case7','case8','case9','case10','case11','case12']) 

In [117]:
data

,case1,case2,case3,case4,case5,case6,case7,case8,case9,case10,case11,case12
0,Frasier,The Andy Griffith Show,Friends,Episodes,The Andy Griffith Show,Episodes,The Andy Griffith Show,Episodes,Friends,Episodes,Cheers,Frasier
1,Episodes,Episodes,The Andy Griffith Show,Pee-wee's Playhouse,Cheers,Highland: Thailand's Marijuana Awakening,Frasier,Frasier,Episodes,Frasier,Frasier,The Andy Griffith Show
2,Dad's Army,Frasier,Cheers,The Twilight Zone (Original Series),Frasier,The Accountant of Auschwitz,Friends,Dad's Army,Cheers,Still Game,The Andy Griffith Show,Cheers
3,The Andy Griffith Show,Friends,Pee-wee's Playhouse,The Andy Griffith Show,Episodes,Miss Representation,Pee-wee's Playhouse,Still Game,The Andy Griffith Show,Dad's Army,Pee-wee's Playhouse,Pee-wee's Playhouse
4,The Twilight Zone (Original Series),Pee-wee's Playhouse,The Twilight Zone (Original Series),Toast of London,Dad's Army,Mountain,The Twilight Zone (Original Series),The Andy Griffith Show,The Twilight Zone (Original Series),The Andy Griffith Show,Jericho,The Twilight Zone (Original Series)
5,Supernatural,The Twilight Zone (Original Series),The Walking Dead,Still Game,Pee-wee's Playhouse,Sustainable,The Walking Dead,Cheers,Pee-wee's Playhouse,Cheers,The Walking Dead,The Walking Dead
6,Still Game,The Walking Dead,Haven,Frasier,The Twilight Zone (Original Series),Studio 54,Jericho,Toast of London,Dad's Army,Toast of London,The Twilight Zone (Original Series),Jericho
7,Pee-wee's Playhouse,Jericho,Jericho,Dad's Army,Jericho,P.S. I Love You,Supernatural,The Twilight Zone (Original Series),Still Game,Pee-wee's Playhouse,"Roswell, New Mexico","Roswell, New Mexico"
8,Cheers,Supernatural,Supernatural,Star Trek: Enterprise,From Dusk Till Dawn,Zoom,From Dusk Till Dawn,The Walking Dead,Toast of London,The Walking Dead,Haven,Supernatural
9,Toast of London,From Dusk Till Dawn,"Roswell, New Mexico",Supernatural,The Walking Dead,The Creative Brain,Haven,Pee-wee's Playhouse,The Walking Dead,Jericho,Supernatural,Haven
